# Initial requirements

This notebook requires IBM Cloud Object Storage and IBM Cloud Functions
Please follow IBM Cloud dashboard and create both services.


In [36]:
# !pip --version

In [9]:
# Install PyWren-IBM into your Conda environment
# Running it right here will use the default Python not the environtment one
# curl -fsSL "https://git.io/fhe9X" | sh

In [10]:
import pywren_ibm_cloud as pywren
pywren.__version__

'1.0.4'

In [16]:
# We need this to overcome Python notebooks limitations of too many open files
import resource
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('Bebore:', soft, hard)

# Raising the soft limit. Hard limits can be raised only by sudo users
resource.setrlimit(resource.RLIMIT_NOFILE, (10000, hard))
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('After:', soft, hard)

Bebore: 1024 1048576
After: 10000 1048576


In [27]:
%config Completer.use_jedi = False
%matplotlib inline

DEBUG:matplotlib:$HOME=/home/intsco
DEBUG:matplotlib:CONFIGDIR=/home/intsco/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /opt/dev/miniconda3/envs/sm/lib/python3.6/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /opt/dev/miniconda3/envs/sm/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.0.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', 

DEBUG:matplotlib:CACHEDIR=/home/intsco/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/intsco/.cache/matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [28]:
from matplotlib import pyplot as plt
from scipy.sparse import coo_matrix
from collections import defaultdict
from pyImagingMSpec.image_measures import isotope_image_correlation, isotope_pattern_match
from cpyImagingMSpec import measure_of_chaos
from itertools import chain
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
import sys
import io

### IBM COS Setup

Setup a bucket in IBM Cloud Object Storage
You need an IBM COS bucket which you will use to store the input data. If you don't know of any of your existing buckets or would like like to create a new one, please navigate to your cloud resource list, then find and select your storage instance. From here, you will be able to view all your buckets and can create a new bucket in the region you prefer. Make sure you copy the correct endpoint for the bucket from the Endpoint tab of this COS service dashboard. Note: The bucket names must be unique.

In [17]:
# Fill here the bucket name you created in COS Dashboard 
#bucket_name = 'kovalev-annotation-example'

In [18]:
# Define COS endpoint information. Example of US Cross Region endpoint
#cos_endpoint = 's3.private.eu-de.cloud-object-storage.appdomain.cloud'

In [19]:
import logging
logging.basicConfig(level=logging.DEBUG)

Obtain the API key and endpoint to the IBM Cloud Functions service. Navigate to Getting Started > API Key from the side menu and copy the values for "Current Namespace", "Host" and "Key" into the config below. Make sure to add "https://" to the host when adding it as the endpoint.

In [20]:
config = {
    'ibm_cf':  {'endpoint': 'https://eu-gb.functions.cloud.ibm.com',
              'namespace': 'kovalev@embl.de_dev', 
              'action_memory' : 1024,
              'api_key': '***'}, 
    'ibm_cos': {
        'endpoint': 'https://s3.eu-gb.cloud-object-storage.appdomain.cloud',
        'api_key' : '***'
    },
    'pywren' : {'storage_bucket' : 'metaspace-annotation-example'}
}

In [21]:
import json

secrets = json.load(open('config.json'))
config['ibm_cf']['api_key'] = secrets['ibm_cf']['api_key']
config['ibm_cos']['api_key'] = secrets['ibm_cos']['api_key']

### IBM Cloud Functions setup

The PyWren engine requires its server side component to be deployed in advance. This step creates a new IBM Cloud Functions function with the PyWren server side runtime. This action will be used internally by PyWren during execution phases.

In [29]:
from pywren_ibm_cloud.deployutil import clone_runtime
clone_runtime('ibmfunctions/pywren-metabolomics:3.6', config, 'pywren-ibm-cloud')

INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for namespace: kovalev@embl.de_dev
INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for host: https://eu-gb.functions.cloud.ibm.com
DEBUG:pywren_ibm_cloud.cf_connector:CF user agent set to: python-requests/2.21.0 pywren-ibm-cloud
DEBUG:pywren_ibm_cloud.cf_connector:I am about to create a new cloud function action


Cloning docker image ibmfunctions/pywren-metabolomics:3.6
OK --> Updated action pywren-metabolomics_3.6


INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for namespace: kovalev@embl.de_dev
INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for host: https://eu-gb.functions.cloud.ibm.com
DEBUG:pywren_ibm_cloud.cf_connector:CF user agent set to: python-requests/2.21.0 pywren-ibm-cloud
DEBUG:pywren_ibm_cloud.cf_connector:I am about to create a new cloud function action


OK --> Updated action pywren-metabolomics_3.6_modules


DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object runtimes/pywren-metabolomics_3.6.meta.json - Size: 7.0KiB - OK


Delete cloud function action: pywren-metabolomics_3.6_modules
All done!


In [30]:
pw = pywren.ibm_cf_executor(config=config, runtime='pywren-metabolomics_3.6')

INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for namespace: kovalev@embl.de_dev
INFO:pywren_ibm_cloud.cf_connector:IBM Cloud Functions init for host: https://eu-gb.functions.cloud.ibm.com
DEBUG:pywren_ibm_cloud.cf_connector:CF user agent set to: python-requests/2.21.0 pywren-ibm-cloud
INFO:pywren_ibm_cloud.invokers:IBM Cloud Functions init for Runtime: pywren-metabolomics_3.6
DEBUG:pywren_ibm_cloud.runtime:Downloading runtime pre-installed modules from COS
DEBUG:pywren_ibm_cloud.runtime:Verifying Python versions
INFO:pywren_ibm_cloud.executor:IBM Cloud Functions executor created with ID a1d4729f-c54d


# Upload test data into COS bucket

In [22]:
import ibm_boto3
from ibm_botocore.client import Config
from ibm_botocore.client import ClientError

In [23]:
cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config['ibm_cos']['api_key'],
#                               ibm_auth_endpoint=config['ibm_cos']['auth_endpoint'],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config['ibm_cos']['endpoint'])

In [24]:
def copy(src, target_bucket, target_key):
    print('Copying from {} to {}/{}'.format(src, target_bucket, target_key))

    with open(src, "rb") as fp:
        cos_client.put_object(Bucket=target_bucket, Key=target_key, Body=fp)

    print('Copy completed for {}/{}'.format(target_bucket, target_key))

In [25]:
import os

for dirpath, dirnames, filenames in os.walk('./metabolomics'):
    for fn in filenames:
        f_path = f'{dirpath}/{fn}'
        copy(f_path, config['pywren']['storage_bucket'], f_path)

Copying from ./metabolomics/db/centroids.csv to metaspace-annotation-example/./metabolomics/db/centroids.csv
Copy completed for metaspace-annotation-example/./metabolomics/db/centroids.csv
Copying from ./metabolomics/db/formulas.csv to metaspace-annotation-example/./metabolomics/db/formulas.csv
Copy completed for metaspace-annotation-example/./metabolomics/db/formulas.csv
Copying from ./metabolomics/input/ds.txt to metaspace-annotation-example/./metabolomics/input/ds.txt
Copy completed for metaspace-annotation-example/./metabolomics/input/ds.txt
Copying from ./metabolomics/input/ds_coord.txt to metaspace-annotation-example/./metabolomics/input/ds_coord.txt
Copy completed for metaspace-annotation-example/./metabolomics/input/ds_coord.txt
Copying from ./metabolomics/data/config.json to metaspace-annotation-example/./metabolomics/data/config.json
Copy completed for metaspace-annotation-example/./metabolomics/data/config.json
Copying from ./metabolomics/data/meta.json to metaspace-annotati

In [26]:
input_data =   {'ds': 'metabolomics/input/ds.txt',
                'ds_coord': 'metabolomics/input/ds_coord.txt',
                'segments': 'metabolomics/input/segments'}
input_db =     {'centroids': 'metabolomics/db/centroids.csv',
                'formulas': 'metabolomics/db/formulas.csv'}
input_config = {'config': 'metabolomics/data/config.json',
                'meta': 'metabolomics/data/meta.json'}

# Read Dataset Spectra

In [37]:
def parse_txt(key, data_stream, func):
    rows = []
    buffer = io.StringIO(data_stream.read().decode('utf-8'))
    while True:
        line = buffer.readline()
        if not line:
            break
        rows.append(func(line))
    return rows

In [38]:
def parse_spectrum_line(s):
    ind_s, mzs_s, int_s = s.split('|')
    return (int(ind_s),
            np.fromstring(mzs_s, sep=' ').astype('float32'),
            np.fromstring(int_s, sep=' '))

In [39]:
def parse_spectrum_coord(s):
    sp_i, x, y = map(int, s.split(','))
    return (sp_i, x, y)

In [40]:
bucket_name = config['pywren']['storage_bucket']
bucket_name

'metaspace-annotation-example'

In [41]:
iterdata = [[f'{bucket_name}/{input_data["ds"]}', parse_spectrum_line]]
# NOTE: we need to be absolutely sure that using chunk_size doesn't split a line into separate chunks
futures = pw.map(parse_txt, iterdata, chunk_size=8*1024**2)
results = pw.get_result(futures)
spectra = []
for res_list in results:
    spectra.extend(res_list)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
INFO:pywren_ibm_cloud.partitioner:Creating chunks from object keys...
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/15e5b2b2/aggdata.pickle - Size: 7.3KiB - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/15e5b2b2/func.pickle - Size: 1.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Starting function invocation: parse_txt() - Total: 11 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00000 invoke
DEBUG:

In [42]:
len(spectra)

6400

In [43]:
sp_i, mzs, ints = spectra[0]
mzs

array([ 53.904255,  55.35594 ,  55.362545,  55.949936,  56.66233 ,
        57.95044 ,  58.101448,  58.48464 ,  58.675667,  59.005196,
        59.035973,  59.161777,  59.415894,  60.146664,  60.161423,
        61.23369 ,  63.52685 ,  63.62095 ,  63.965775,  64.112434,
        65.029106,  65.40917 ,  65.61097 ,  66.66296 ,  66.77625 ,
        66.88279 ,  67.84289 ,  68.00082 ,  69.09414 ,  69.1053  ,
        70.70261 ,  71.01698 ,  71.21804 ,  71.29459 ,  71.79796 ,
        72.849205,  72.885   ,  74.43753 ,  74.84822 ,  74.984146,
        75.33112 ,  81.7692  ,  90.1316  ,  90.21486 ,  91.43951 ,
        94.44622 ,  95.16278 ,  95.54628 ,  96.42089 ,  99.29104 ,
       100.45906 , 100.77831 , 105.90295 , 111.90609 , 112.87558 ,
       115.86939 , 119.84346 , 122.13882 , 125.892654, 129.55223 ,
       135.2007  , 137.09537 , 137.28242 , 137.34137 , 138.45294 ,
       145.36913 , 145.76651 , 146.5534  , 159.07352 , 160.38416 ,
       161.22887 , 163.37315 , 163.4697  , 167.33083 , 168.375

In [44]:
ints

array([47.29862213, 50.59011841, 47.02101135, 47.47746277, 58.65240479,
       50.92343521, 42.76822662, 42.45158768, 56.05449677, 43.1148262 ,
       48.53097153, 55.81335449, 50.67507553, 43.09651566, 46.87289047,
       54.37233353, 47.54389572, 54.30453873, 47.79224777, 46.20689392,
       50.62897873, 40.02255631, 46.42947006, 43.94950485, 44.11400986,
       49.48485565, 49.11848068, 48.14740372, 40.9396019 , 41.03996277,
       44.44740295, 49.41588211, 58.40541458, 50.03430557, 43.44033432,
       48.588768  , 57.39586258, 53.97465515, 54.93380356, 84.99694061,
       49.48104477, 56.77067566, 51.75605393, 43.9488945 , 54.65294266,
       56.53967667, 60.78800964, 57.96722794, 52.62657928, 55.60599518,
       49.46104431, 58.96443176, 56.40863037, 70.75928497, 62.49911881,
       62.38255692, 52.94942093, 52.0222168 , 52.91330719, 56.34723282,
       49.91038132, 68.52725983, 56.43950272, 64.62987518, 68.28511047,
       58.55063629, 56.82152176, 65.20709992, 57.73368454, 58.89

In [45]:
iterdata = [[f'{bucket_name}/{input_data["ds_coord"]}', parse_spectrum_coord]]
futures = pw.map(parse_txt, iterdata)
spectra_coords = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds_coord.txt
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/2b4a3253/aggdata.pickle - Size: 560.0B - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/2b4a3253/func.pickle - Size: 1.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Starting function invocation: parse_txt() - Total: 1 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00000 invoke
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID a1d4729f-c54d Function 

In [46]:
len(spectra_coords)

6400

In [47]:
spectra_coords[:5]

[(0, 1, 1), (1, 2, 1), (2, 3, 1), (3, 4, 1), (4, 5, 1)]

# Read Molecular Database

In [48]:
def process_formulas_database(key, data_stream):
    formulas_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    return formulas_df.shape, formulas_df.head()

In [49]:
iterdata = [f'{bucket_name}/{input_db["formulas"]}']
futures = pw.map(process_formulas_database, iterdata)
formulas_shape, formulas_head = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/db/formulas.csv
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/22a55d1b/aggdata.pickle - Size: 138.0B - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/22a55d1b/func.pickle - Size: 1.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Starting function invocation: process_formulas_database() - Total: 1 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00000 invoke
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID a1d4729f-c

In [50]:
formulas_shape

(1149239, 1)

In [51]:
formulas_head

,formula
formula_i,
0,C100H121N50O58P9Al
1,C100H121N50O58P9As
2,C100H121N50O58P9Au
3,C100H121N50O58P9B
4,C100H121N50O58P9Ba


In [52]:
def process_centroids_database(key, data_stream):
    centroids_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    return centroids_df.shape, centroids_df.head(8)

In [53]:
%%time
iterdata = [f'{bucket_name}/{input_db["centroids"]}']
futures = pw.map(process_centroids_database, iterdata)
centroids_shape, centroids_head = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/db/centroids.csv
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/4fc341bb/aggdata.pickle - Size: 139.0B - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/4fc341bb/func.pickle - Size: 1.6KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Starting function invocation: process_centroids_database() - Total: 1 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00000 invoke
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID a1d4729f

CPU times: user 140 ms, sys: 12 ms, total: 152 ms
Wall time: 9.17 s


In [54]:
centroids_shape

(4596786, 3)

In [55]:
centroids_head

,peak_i,mz,int
formula_i,,,
0,0,3255.550401,91.732506
0,1,3256.553764,100.000000
0,2,3257.557145,54.004425
0,3,3258.560492,19.359366
1,0,3303.490459,91.712252
1,1,3304.493826,100.000000
1,2,3305.497206,53.990998
1,3,3306.500550,19.354821


# Generate Ion Images

In [56]:
def real_pixel_indices(spectra_coords):
    coord_pairs = [r[1:] for r in spectra_coords]

    min_x, min_y = np.amin(np.asarray(coord_pairs), axis=0)
    max_x, max_y = np.amax(np.asarray(coord_pairs), axis=0)

    _coord = np.array(coord_pairs)
    _coord = np.around(_coord, 5)  # correct for numerical precision
    _coord -= np.amin(_coord, axis=0)

    nrows, ncols = (max_y - min_y + 1,
                    max_x - min_x + 1)

    pixel_indices = _coord[:, 1] * ncols + _coord[:, 0]
    return pixel_indices.astype(np.int32), nrows, ncols

In [57]:
def sp_df_gen(sp_it, pixel_indices):
    for sp_id, mzs, intensities in sp_it:
        for mz, ints in zip(mzs, intensities):
            yield pixel_indices[sp_id], mz, ints

In [58]:
def gen_iso_images(spectra_it, pixel_indices, centr_df, nrows, ncols, ppm, min_px=1):
    if len(centr_df) > 0:
        # a bit slower than using pure numpy arrays but much shorter
        # may leak memory because of https://github.com/pydata/pandas/issues/2659 or smth else
        sp_df = pd.DataFrame(sp_df_gen(spectra_it, pixel_indices),
                             columns=['idx', 'mz', 'ints']).sort_values(by='mz')

        # -1, + 1 are needed to extend sf_peak_mz range so that it covers 100% of spectra
        centr_df = centr_df[(centr_df.mz >= sp_df.mz.min() - 1) &
                            (centr_df.mz <= sp_df.mz.max() + 1)]
        lower = centr_df.mz.map(lambda mz: mz - mz * ppm * 1e-6)
        upper = centr_df.mz.map(lambda mz: mz + mz * ppm * 1e-6)
        lower_idx = np.searchsorted(sp_df.mz, lower, 'l')
        upper_idx = np.searchsorted(sp_df.mz, upper, 'r')

        for i, (l, u) in enumerate(zip(lower_idx, upper_idx)):
            if u - l >= min_px:
                data = sp_df.ints[l:u].values
                if data.shape[0] > 0:
                    idx = sp_df.idx[l:u].values
                    row_inds = idx / ncols
                    col_inds = idx % ncols
                    m = coo_matrix((data, (row_inds, col_inds)), shape=(nrows, ncols), copy=True)
                    yield centr_df.index[i], (centr_df.peak_i.iloc[i], m)

In [59]:
def merge_formula_images(iso_images):
    formula_images = defaultdict(list)
    for formula_i, (peak_i, img) in iso_images:
        formula_images[formula_i].append((peak_i, img))

    def filter_formula_images():
        filtered_f_images = {}
        for f_i, images in formula_images.items():
            images = sorted(images, key=lambda x: x[0])
            if len(images) > 1 and images[0][0] == 0:
                filtered_f_images[f_i] = images
        return filtered_f_images

    return filter_formula_images()

In [60]:
pixel_indices, nrows, ncols = real_pixel_indices(spectra_coords)

In [61]:
pixel_indices

array([   0,    1,    2, ..., 6397, 6398, 6399], dtype=int32)

In [62]:
nrows, ncols

(80, 80)

# Score Formula Images

In [63]:
def score_formula_images(f_images, centroids_df, empty_image):
    formula_scores = []
    for formula_i, images in f_images.items():
        centr_ints = centroids_df.loc[formula_i].int.values

        image_list = [empty_image] * len(centr_ints)
        for peak_i, img in images:
            image_list[peak_i] = img.toarray()
        flat_image_list = [img.flat[:] for img in image_list]

        m1 = isotope_pattern_match(flat_image_list, centr_ints)
        m2 = isotope_image_correlation(flat_image_list, centr_ints[1:])
        m3 = measure_of_chaos(image_list[0], nlevels=30)
        formula_scores.append([formula_i, m1, m2, m3])

    formula_scores_df = pd.DataFrame(formula_scores,
                                     columns=['formula_i', 'm1', 'm2', 'm3']).set_index('formula_i')
    formula_scores_df['msm'] = formula_scores_df.m1 * formula_scores_df.m2 * formula_scores_df.m3
    return formula_scores_df

In [64]:
empty_image = np.zeros((nrows, ncols))

In [65]:
empty_image.shape

(80, 80)

# Split Dataset into Segments

In [66]:
segm_n = 256

In [67]:
def get_segm_intervals(key, data_stream):
    centroids_df = pd.read_csv(data_stream._raw_stream).set_index('formula_i')
    segm_bounds_q = [i * 1 / segm_n for i in range(1, segm_n)]
    segm_bounds = [np.quantile(centroids_df.mz.values, q) for q in segm_bounds_q]
    segm_bounds = [0.] + segm_bounds + [sys.float_info.max]
    segm_intervals = list(zip(segm_bounds[:-1], segm_bounds[1:]))
    return segm_intervals

In [68]:
iterdata = [f'{bucket_name}/{input_db["centroids"]}']
futures = pw.map(get_segm_intervals, iterdata)
segm_intervals = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/db/centroids.csv
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Serializing function and data
DEBUG:pywren_ibm_cloud.serialize.serialize:Modules to transmit: None
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Uploading function and data
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/cbdaf77d/aggdata.pickle - Size: 139.0B - OK
DEBUG:pywren_ibm_cloud.storage.backends.cos:PUT Object pywren.jobs/a1d4729f-c54d/cbdaf77d/func.pickle - Size: 2.0KiB - OK
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Starting function invocation: get_segm_intervals() - Total: 1 activations
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00000 invoke
DEBUG:pywren_ibm_cloud.cf_connector:Executor ID a1d4729f-c54d Fu

In [69]:
segm_intervals[:5]

[(0.0, 116.0661353472162),
 (116.0661353472162, 138.02771722325699),
 (138.02771722325699, 152.9807543117649),
 (152.9807543117649, 164.93990146661145),
 (164.93990146661145, 174.8942965607513)]

In [70]:
def iterate_over_segment(spectra, min_mz, max_mz):
    for sp_i, mzs, ints in spectra:
        smask = (mzs >= min_mz) & (mzs <= max_mz)
        yield [sp_i], mzs[smask], ints[smask]

In [71]:
def store_segm(key, data_stream, ibm_cos, segm_i, interval):
    spectra = parse_txt(key, data_stream, parse_spectrum_line)
    segm_it = iterate_over_segment(spectra, *interval)
    segm_spectra = pickle.dumps(np.array(list(segm_it)))
    ibm_cos.put_object(Bucket=bucket_name,
                       Key=f'{input_data["segments"]}/{segm_i}.pickle',
                       Body=segm_spectra)

In [72]:
%%time
iterdata = [[f"{bucket_name}/{input_data['ds']}", segm_i, segm_intervals[segm_i]]
             for segm_i in range(segm_n)]
futures = pw.map(store_segm, iterdata)
results = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-examp

INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud

INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/ds.txt
INFO:pywren_ibm_cloud

DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00015 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00016 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00017 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00018 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00019 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00020 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00021 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00022 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00023 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00024 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00025 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00026 invoke
DEBUG:pywren_ibm

DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00115 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00116 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00117 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00118 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00119 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00120 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00121 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00122 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00123 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00124 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00125 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00126 invoke
DEBUG:pywren_ibm

...

DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00169 - Activation ID: 349342566d2043659342566d20136514 - Time: 7.28 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00170 - Activation ID: 660294e5b21542358294e5b215123528 - Time: 7.04 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00171 - Activation ID: ef7291e6c72d4983b291e6c72d7983f3 - Time: 6.86 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00172 - Activation ID: aba55483a4844412a55483a484d412d7 - Time: 6.93 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00173 - Activation ID: 550ac3dc654247208ac3dc6542d720d2 - Time: 6.96 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00174 - Activation ID: bcd9f2de0e904be499f2de0e909be454 - Time: 6.52 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response

DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00164 - Activation ID: 79f85292137a4acfb85292137a9acf17
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00162 - Activation ID: 388558dc21b24f5a8558dc21b2bf5add
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00165 - Activation ID: 85ce34b0cfe2436c8e34b0cfe2936c73
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00163 - Activation ID: 538ef4f8436b44ac8ef4f8436b84acd6
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00167 - Activation ID: 2af49e43f5f2476db49e43f5f2776d8c
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00166 - Activation ID: e5346d8689ec44e3b46d8689ec04e3cd
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00168 - Activation ID: 928a846346d84a388a846346d88a385a
DEBUG:pywren_ibm_cloud.future:Executor ID

DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00219 - Activation ID: e0ffd66a83e145eebfd66a83e105eef2 - Time: 6.36 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00220 - Activation ID: 0bea44652da44104aa44652da47104ff - Time: 7.14 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00221 - Activation ID: cb9700d0bad744429700d0bad7c442d2 - Time: 7.10 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00222 - Activation ID: 8e07b4680c19428187b4680c1962815a - Time: 7.01 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00224 - Activation ID: fbcb96fb0e724ab98b96fb0e720ab9c6 - Time: 6.38 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00223 - Activation ID: b652ed2653ef4e0c92ed2653ef0e0c94 - Time: 6.51 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response

DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00212 - Activation ID: 86d71ace1b3049e1971ace1b30d9e1d5
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00214 - Activation ID: e32e224451714fa5ae224451711fa5e7
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00215 - Activation ID: a08a347dce7e4b868a347dce7eeb8631
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00216 - Activation ID: 3d534c0958be475c934c0958bee75cbe
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00217 - Activation ID: f1b6395447924685b639544792968582
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00218 - Activation ID: c813553bf2854a1293553bf2859a12fe
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00219 - Activation ID: e0ffd66a83e145eebfd66a83e105eef2
DEBUG:pywren_ibm_cloud.future:Executor ID

CPU times: user 4.71 s, sys: 440 ms, total: 5.15 s
Wall time: 1min 15s


# Annotation Pipeline Applied to each Segment in Parallel

In [73]:
def filter_formula_images(formula_images, formula_scores_df):
    return {f_i: images
            for (f_i, images) in formula_images.items()
            if f_i in formula_scores_df.index}

In [74]:
def annotate_segm_spectra(key, data_stream, ibm_cos):
    print("Welcome to annotate spectra")
    centroids_stream = ibm_cos.get_object(Bucket=bucket_name, Key=input_db['centroids'])['Body']
    print("Read centroids DB from IBM COS")
    centroids_df = pd.read_csv(centroids_stream).set_index('formula_i')
    print("Create Pandas DF for centroids DB")
    spectra = pickle.loads(data_stream.read())
    iso_images = gen_iso_images(spectra, pixel_indices, centroids_df, nrows, ncols, ppm=3)
    formula_images = merge_formula_images(list(iso_images))
    formula_scores_df = score_formula_images(formula_images, centroids_df, empty_image)
    formula_scores_df = formula_scores_df[formula_scores_df.msm > 0]
    formula_images = filter_formula_images(formula_images, formula_scores_df)
    return formula_scores_df, formula_images 

In [75]:
%%time
iterdata = [f'{bucket_name}/{input_data["segments"]}/{segm_i}.pickle' for segm_i in range(segm_n)]
futures = pw.map(annotate_segm_spectra, iterdata)
results = pw.get_result(futures)

DEBUG:pywren_ibm_cloud.executor:Calling map on partitions from object storage flow
DEBUG:pywren_ibm_cloud.partitioner:Starting partitioner
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/0.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/1.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/2.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/3.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/4.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/5.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/6.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/7.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-exampl

INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/80.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/81.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/82.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/83.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/84.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/85.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/86.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/87.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/88.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/

INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/161.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/162.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/163.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/164.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/165.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/166.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/167.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/168.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/169.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/

INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/242.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/243.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/244.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/245.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/246.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/247.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/248.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/249.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/segments/250.pickle
INFO:pywren_ibm_cloud.partitioner:metaspace-annotation-example/metabolomics/input/

DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00074 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00075 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00076 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00077 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00078 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00079 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00080 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00081 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00082 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00083 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00084 invoke
DEBUG:pywren_ibm_cloud.executor:Executor ID a1d4729f-c54d Activation 00085 invoke
DEBUG:pywren_ibm

...

DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00134 - Activation ID: 8c7971d55ac4460fb971d55ac4460f6c
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00059 - Activation ID: 25ffbc5c45c9466fbfbc5c45c9666fae
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00112 - Activation ID: d51a487e7e9646379a487e7e96a63709
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00063 - Activation ID: 389a5c2a7dff49c79a5c2a7dff99c797 - Time: 27.59 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00070 - Activation ID: 96c38d19c0c844f8838d19c0c8e4f8c0 - Time: 28.32 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00076 - Activation ID: 2d37915be4bc4e35b7915be4bc7e35d3 - Time: 26.83 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00080 - Activation ID: cedfd783d5e9494b

DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00051 - Activation ID: 5db9aaa82f1d4c22b9aaa82f1dac2229
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00097 - Activation ID: d9fbd6b2107e4a99bbd6b2107e8a997b
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00141 - Activation ID: b022942052504842a294205250e84265
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00130 - Activation ID: c74ee5f8134649ed8ee5f81346e9ed43
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00138 - Activation ID: 67c454fce73744388454fce7378438e8
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00049 - Activation ID: a024f80b4c8448a8a4f80b4c84b8a880
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00078 - Activation ID: 1147494f30a84be087494f30a86be0a1
DEBUG:pywren_ibm_cloud.future:Executor ID

DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00046 - Activation ID: 44d7af9c3693447797af9c369324776e - Time: 32.67 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00036 - Activation ID: 5ea1cafdcf334c3ba1cafdcf333c3b2f - Time: 33.09 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00047 - Activation ID: 0818cf5e88ac44ef98cf5e88ac14efc0 - Time: 32.83 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00048 - Activation ID: eec23b46295a4be2823b46295a7be299 - Time: 33.23 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00052 - Activation ID: 7aa75f569b8445f3a75f569b8465f39d - Time: 33.74 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00029 - Activation ID: 72c5616adb31425085616adb31725003 - Time: 34.40 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Go

DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Got output from Function 00017 - Activation ID: 87342cf448114538b42cf44811953892
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00006 - Activation ID: 92d10ca52f1c4c51910ca52f1cbc512d - Time: 43.72 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00010 - Activation ID: f940da52587149b580da52587139b544 - Time: 44.20 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00013 - Activation ID: 9f6810ddcd2c4f77a810ddcd2c1f77d9 - Time: 45.97 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00009 - Activation ID: 6fd78d4384c149bc978d4384c1b9bcb0 - Time: 44.73 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function 00011 - Activation ID: f9417df95ad6451d817df95ad6651d82 - Time: 45.86 seconds
DEBUG:pywren_ibm_cloud.future:Executor ID a1d4729f-c54d Response from Function

ReadTimeoutError: Read timeout on endpoint URL: "None"

In [76]:
len(results)

256

# Get Results

In [77]:
formula_scores_list, formula_images_list = list(zip(*results))
formula_scores_df = pd.concat(formula_scores_list)
formula_images = dict(chain(*[segm_formula_images.items()
                              for segm_formula_images in formula_images_list]))

TypeError: zip argument #1 must support iteration

In [ ]:
img = formula_images[896952][0][1]
plt.imshow(img.toarray())

## Clean Segments Datasets

In [ ]:
def clean_segments_datasets(bucket, key, data_stream, ibm_cos):
    ibm_cos.delete_object(Bucket=bucket, Key=key)

In [ ]:
%%time
data_stream = f'{bucket_name}/{input_data["segments"]}'
futures = pw.map(clean_segments_datasets, data_stream)
pw.get_result(futures)

In [ ]:
pw.clean()